In [ ]:
df_geo = df_clean[(df_clean['Latitude'].notna()) & (df_clean['GHG_Total'] > 0)].copy()

fig13 = px.scatter_mapbox(
    df_geo,
    lat='Latitude',
    lon='Longitude',
    size='GHG_Total',
    color='GHG_Intensity',
    hover_data=['Property Name', 'Property_Type', 'Borough', 'Site_EUI'],
    title='Geographic Distribution of GHG Emissions Across NYC',
    color_continuous_scale='Reds',
    size_max=15,
    zoom=10,
    mapbox_style='carto-positron'
)
fig13.update_layout(height=700)
fig13.show()

# ============================================================================
# 14. Borough Comparison - Performance Metrics
# ============================================================================
borough_stats = df_clean.groupby('Borough').agg({
    'Site_EUI': ['mean', 'median'],
    'GHG_Intensity': ['mean', 'median'],
    'ENERGY_STAR': 'mean',
    'Property ID': 'count'
}).round(2)
borough_stats.columns = ['_'.join(col).strip() for col in borough_stats.columns]
borough_stats = borough_stats.reset_index()

fig14 = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Avg Site EUI by Borough', 'Avg GHG Intensity by Borough',
                    'Avg ENERGY STAR Score by Borough', 'Building Count by Borough')
)

fig14.add_trace(
    go.Bar(x=borough_stats['Borough'], y=borough_stats['Site_EUI_mean'], name='Site EUI'),
    row=1, col=1
)
fig14.add_trace(
    go.Bar(x=borough_stats['Borough'], y=borough_stats['GHG_Intensity_mean'], name='GHG Intensity'),
    row=1, col=2
)
fig14.add_trace(
    go.Bar(x=borough_stats['Borough'], y=borough_stats['ENERGY_STAR_mean'], name='ENERGY STAR'),
    row=2, col=1
)
fig14.add_trace(
    go.Bar(x=borough_stats['Borough'], y=borough_stats['Property ID_count'], name='Count'),
    row=2, col=2
)

fig14.update_layout(height=800, showlegend=False, title_text='Borough Performance Comparison')
fig14.show()

# ============================================================================
# 15. Building Age Cohort Analysis
# ============================================================================
# Create age bins
df_age = df_clean[df_clean['Year Built'] > 1800].copy()
bins = [1800, 1900, 1920, 1945, 1960, 1980, 2000, 2025]
labels = ['Pre-1900', '1900-1920', '1920-1945', '1945-1960', '1960-1980', '1980-2000', '2000+']
df_age['Age_Cohort'] = pd.cut(df_age['Year Built'], bins=bins, labels=labels)

age_performance = df_age.groupby('Age_Cohort').agg({
    'Site_EUI': 'mean',
    'GHG_Intensity': 'mean',
    'ENERGY_STAR': 'mean',
    'Property ID': 'count'
}).reset_index()

fig15 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('EUI by Building Age Cohort', 'GHG Intensity by Age Cohort')
)

fig15.add_trace(
    go.Bar(x=age_performance['Age_Cohort'], y=age_performance['Site_EUI'], 
           name='Site EUI', marker_color='steelblue'),
    row=1, col=1
)
fig15.add_trace(
    go.Bar(x=age_performance['Age_Cohort'], y=age_performance['GHG_Intensity'], 
           name='GHG Intensity', marker_color='coral'),
    row=1, col=2
)

fig15.update_layout(height=500, showlegend=False, title_text='Energy Performance by Building Age Cohort')
fig15.show()

# ============================================================================
# 16. Peer Group Benchmarking (Percentile Analysis)
# ============================================================================
# For each property type, calculate percentiles
def calculate_percentile(group):
    group['EUI_Percentile'] = group['Site_EUI'].rank(pct=True) * 100
    group['GHG_Percentile'] = group['GHG_Intensity'].rank(pct=True) * 100
    return group

df_benchmark = df_clean[df_clean['Property_Type'].isin(top_types)].copy()
df_benchmark = df_benchmark.groupby('Property_Type').apply(calculate_percentile)

fig16 = px.scatter(
    df_benchmark[df_benchmark['Site_EUI'] > 0],
    x='EUI_Percentile',
    y='GHG_Percentile',
    color='Property_Type',
    hover_data=['Property Name', 'Site_EUI', 'GHG_Intensity'],
    title='Peer Group Benchmarking: EUI vs GHG Percentiles Within Property Type',
    labels={'EUI_Percentile': 'EUI Percentile Within Property Type (%)',
            'GHG_Percentile': 'GHG Intensity Percentile Within Property Type (%)'}
)
fig16.add_shape(type="line", x0=50, y0=0, x1=50, y1=100, line=dict(dash="dash", color="gray"))
fig16.add_shape(type="line", x0=0, y0=50, x1=100, y1=50, line=dict(dash="dash", color="gray"))
fig16.update_layout(height=600)
fig16.show()

# ============================================================================
# 17. Weather Normalization Impact Analysis
# ============================================================================
df_weather = df_clean[(df_clean['Site_EUI'] > 0) & (df_clean['Weather_Norm_EUI'] > 0)].copy()
df_weather['Weather_Impact'] = ((df_weather['Weather_Norm_EUI'] - df_weather['Site_EUI']) / 
                                 df_weather['Site_EUI'] * 100)

fig17 = px.histogram(
    df_weather[abs(df_weather['Weather_Impact']) < 50],
    x='Weather_Impact',
    nbins=50,
    title='Weather Normalization Impact on EUI (%)',
    labels={'Weather_Impact': 'Weather Impact (% change from actual to normalized)'}
)
fig17.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="No Impact")
fig17.update_layout(height=500)
fig17.show()

# ============================================================================
# 18. Energy vs Emissions Efficiency (Decoupling Analysis)
# ============================================================================
df_efficiency = df_clean[(df_clean['Site_Energy'] > 0) & (df_clean['GHG_Total'] > 0)].copy()
df_efficiency['Emissions_per_Energy'] = df_efficiency['GHG_Total'] / df_efficiency['Site_Energy'] * 1000

fig18 = px.box(
    df_efficiency[df_efficiency['Property_Type'].isin(top_types)],
    x='Property_Type',
    y='Emissions_per_Energy',
    title='Carbon Intensity of Energy Use by Property Type (Emissions per Unit Energy)',
    labels={'Emissions_per_Energy': 'kg CO2e per Million kBtu', 'Property_Type': 'Property Type'},
    color='Property_Type'
)
fig18.update_layout(height=600, showlegend=False)
fig18.update_xaxes(tickangle=45)
fig18.show()

# ============================================================================
# 19. Size Category Analysis
# ============================================================================
# Categorize buildings by size
df_size = df_clean[df_clean['GFA'] > 0].copy()
df_size['Size_Category'] = pd.cut(df_size['GFA'], 
                                   bins=[0, 25000, 50000, 100000, 250000, float('inf')],
                                   labels=['Small (<25k)', 'Medium (25-50k)', 
                                          'Large (50-100k)', 'Very Large (100-250k)', 
                                          'Mega (>250k)'])

size_stats = df_size.groupby('Size_Category').agg({
    'Site_EUI': 'mean',
    'GHG_Intensity': 'mean',
    'Property ID': 'count'
}).reset_index()

fig19 = make_subplots(
    rows=1, cols=2,
    specs=[[{"secondary_y": True}, {"type": "bar"}]],
    subplot_titles=('Performance by Building Size', 'Building Count by Size')
)

fig19.add_trace(
    go.Bar(x=size_stats['Size_Category'], y=size_stats['Site_EUI'], 
           name='Site EUI', marker_color='steelblue'),
    row=1, col=1
)
fig19.add_trace(
    go.Scatter(x=size_stats['Size_Category'], y=size_stats['GHG_Intensity'], 
               name='GHG Intensity', mode='lines+markers', 
               marker_color='red', line=dict(width=3)),
    row=1, col=1, secondary_y=True
)
fig19.add_trace(
    go.Bar(x=size_stats['Size_Category'], y=size_stats['Property ID'], 
           name='Count', marker_color='green'),
    row=1, col=2
)

fig19.update_xaxes(title_text="Size Category", row=1, col=1)
fig19.update_xaxes(title_text="Size Category", row=1, col=2)
fig19.update_yaxes(title_text="Site EUI", row=1, col=1)
fig19.update_yaxes(title_text="GHG Intensity", row=1, col=1, secondary_y=True)
fig19.update_yaxes(title_text="Number of Buildings", row=1, col=2)

fig19.update_layout(height=500, title_text='Building Size Category Analysis')
fig19.show()

# ============================================================================
# 20. Electrification Index
# ============================================================================
df_fuel_mix = df_clean[(df_clean['Electricity'] > 0) & (df_clean['Gas'] > 0)].copy()
# Calculate electrification ratio (electricity kBtu / total energy kBtu)
df_fuel_mix['Elec_kBtu'] = df_fuel_mix['Electricity'] * 3.412
df_fuel_mix['Gas_kBtu'] = df_fuel_mix['Gas'] * 100
df_fuel_mix['Electrification_Ratio'] = (df_fuel_mix['Elec_kBtu'] / 
                                         (df_fuel_mix['Elec_kBtu'] + df_fuel_mix['Gas_kBtu']))

fig20 = px.histogram(
    df_fuel_mix,
    x='Electrification_Ratio',
    color='Property_Type',
    nbins=30,
    title='Electrification Ratio Distribution by Property Type (0=All Gas, 1=All Electric)',
    labels={'Electrification_Ratio': 'Electrification Ratio'},
    marginal='box'
)
fig20.update_layout(height=600)
fig20.show()

# ============================================================================
# 21. ENERGY STAR Score Gap Analysis
# ============================================================================
df_star_gap = df_clean[(df_clean['ENERGY_STAR'] > 0)].copy()
df_star_gap['Certified'] = df_star_gap['ENERGY_STAR'] >= 75

star_comparison = df_star_gap.groupby(['Property_Type', 'Certified']).agg({
    'Site_EUI': 'mean',
    'GHG_Intensity': 'mean',
    'Property ID': 'count'
}).reset_index()

# Filter to types with enough data
type_counts = star_comparison.groupby('Property_Type')['Property ID'].sum()
valid_types = type_counts[type_counts >= 20].index
star_comparison = star_comparison[star_comparison['Property_Type'].isin(valid_types)]

fig21 = px.bar(
    star_comparison.head(20),
    x='Property_Type',
    y='Site_EUI',
    color='Certified',
    barmode='group',
    title='EUI: ENERGY STAR Certified (≥75) vs Not Certified (<75)',
    labels={'Site_EUI': 'Average Site EUI', 'Certified': 'ENERGY STAR Certified'},
    color_discrete_map={True: 'green', False: 'red'}
)
fig21.update_layout(height=600)
fig21.update_xaxes(tickangle=45)
fig21.show()

# ============================================================================
# 22. Outlier Detection - Mahalanobis Distance
# ============================================================================
from scipy.spatial.distance import mahalanobis
from scipy import stats

# Select features for outlier detection
outlier_features = ['Site_EUI', 'GHG_Intensity', 'ENERGY_STAR']
df_outlier = df_clean[outlier_features].dropna()

# Standardize data
df_standard = (df_outlier - df_outlier.mean()) / df_outlier.std()

# Calculate Mahalanobis distance
mean = df_standard.mean().values
cov = df_standard.cov().values
df_outlier['Mahalanobis'] = df_standard.apply(
    lambda row: mahalanobis(row.values, mean, np.linalg.inv(cov)), axis=1
)

# Flag outliers (p-value < 0.001)
df_outlier['p_value'] = 1 - stats.chi2.cdf(df_outlier['Mahalanobis']**2, df=len(outlier_features))
df_outlier['Is_Outlier'] = df_outlier['p_value'] < 0.001

# Merge back with original data
df_clean_outlier = df_clean.loc[df_outlier.index].copy()
df_clean_outlier['Is_Outlier'] = df_outlier['Is_Outlier']

fig22 = px.scatter(
    df_clean_outlier,
    x='Site_EUI',
    y='GHG_Intensity',
    color='Is_Outlier',
    hover_data=['Property Name', 'Property_Type', 'Borough', 'ENERGY_STAR'],
    title='Multivariate Outlier Detection (Mahalanobis Distance)',
    labels={'Is_Outlier': 'Statistical Outlier'},
    color_discrete_map={True: 'red', False: 'blue'}
)
fig22.update_layout(height=600)
fig22.show()

print(f"\n🔍 Identified {df_outlier['Is_Outlier'].sum()} statistical outliers ({df_outlier['Is_Outlier'].sum()/len(df_outlier)*100:.1f}%)")

# ============================================================================
# 23. Emissions Reduction Potential Analysis
# ============================================================================
# Calculate potential savings if all buildings reached median performance for their type
df_potential = df_clean[df_clean['GHG_Total'] > 0].copy()

# Get median GHG intensity for each property type
median_by_type = df_potential.groupby('Property_Type')['GHG_Intensity'].median()
df_potential['Median_GHG_Intensity'] = df_potential['Property_Type'].map(median_by_type)

# Calculate potential emissions if at median
df_potential['Potential_GHG'] = df_potential['GFA'] * df_potential['Median_GHG_Intensity'] / 1000
df_potential['Savings_Potential'] = df_potential['GHG_Total'] - df_potential['Potential_GHG']
df_potential['Savings_Potential'] = df_potential['Savings_Potential'].clip(lower=0)

# Aggregate by property type
savings_by_type = df_potential.groupby('Property_Type').agg({
    'Savings_Potential': 'sum',
    'GHG_Total': 'sum'
}).reset_index()
savings_by_type['Percent_Reduction'] = (savings_by_type['Savings_Potential'] / 
                                         savings_by_type['GHG_Total'] * 100)
savings_by_type = savings_by_type.nlargest(15, 'Savings_Potential')

fig23 = go.Figure(data=[
    go.Bar(name='Current Emissions', x=savings_by_type['Property_Type'], 
           y=savings_by_type['GHG_Total'], marker_color='red'),
    go.Bar(name='Reduction Potential', x=savings_by_type['Property_Type'], 
           y=savings_by_type['Savings_Potential'], marker_color='green')
])
fig23.update_layout(
    title='GHG Emissions Reduction Potential by Property Type (If All Reach Median)',
    xaxis_title='Property Type',
    yaxis_title='Total Emissions (Metric Tons CO2e)',
    barmode='stack',
    height=600
)
fig23.update_xaxes(tickangle=45)
fig23.show()

total_potential = df_potential['Savings_Potential'].sum()
total_current = df_potential['GHG_Total'].sum()
print(f"\n💡 Total GHG Reduction Potential: {total_potential:,.0f} Metric Tons CO2e")
print(f"   ({total_potential/total_current*100:.1f}% reduction if all buildings reach median performance)")

print("\n✅ All 23 visualizations completed!")
print("\n📊 Additional Advanced Analyses Summary:")
print("13. Geospatial map - identifies emission hotspots")
print("14. Borough comparison - performance across NYC")
print("15. Age cohort analysis - vintage effects on energy use")
print("16. Peer benchmarking - percentile ranking within building types")
print("17. Weather normalization - climate impact quantification")
print("18. Carbon intensity - emissions per energy unit")
print("19. Size category analysis - scale effects")
print("20. Electrification index - fuel mix distribution")
print("21. ENERGY STAR gap analysis - certification benefits")
print("22. Outlier detection - multivariate statistical anomalies")
print("23. Reduction potential - quantified savings opportunity")